<a href="https://colab.research.google.com/github/Kussil/Financial_Sentiment_LLM/blob/main/03_Sentiment_Analysis/FinBERT/Sentiment_Analysis_(FinBERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FinBERT Sentiment Analysis

In [ ]:
#installs
!pip install -q tqdm
!pip install -q transformers

In [ ]:
#imports
import os
from google.colab import userdata
import pandas as pd
import math
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch
from tqdm import tqdm

### Import Files From GitHub

In [ ]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

Cloning into 'Financial_Sentiment_LLM'...
remote: Enumerating objects: 2167, done.
remote: Counting objects: 100% (572/572), done.
remote: Compressing objects: 100% (260/260), done.
remote: Total 2167 (delta 343), reused 494 (delta 300), pack-reused 1595
Receiving objects: 100% (2167/2167), 472.57 MiB | 5.72 MiB/s, done.
Resolving deltas: 100% (1426/1426), done.
Updating files: 100% (1142/1142), done.


In [ ]:
invest_df1 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/SEC_Filings.csv')

df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
df['Article Index'] = df.index
df['Date'] = pd.to_datetime(df['Date'], format='mixed')
df['Date']= pd.to_datetime(df['Date']).dt.date
df

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN,1
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN,2
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN,3
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN,4
...,...,...,...,...,...,...,...,...
10048,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN,10048
10049,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10049
10050,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN,10050
10051,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN,10051


### Chunk Text for FinBERT, < 512 token input

In [ ]:
#Chunk text
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')

#Set token limit based on model constraint - BERT is 512
token_limit = 500

#Create function to chunk data
def chunk_contents(x):
  '''
  This function chunks the inputs from a pandas dataframe based on a provided
  token limit. For FinBert this limit is set to 500 because BERT max is 512
  and decoding creates additional tokens

  Parameters:
  x (str): Article Text from a pandas dataframe of all the financial documents

  Returns
  chunk_list (list of str): A python list of the Article Text broken into
  chunks, thes chunks are in a list inside a single row of the source dataframe
  '''
  x_str = str(x)
  tokens = tokenizer(x_str)['input_ids']
  token_count = len(tokens)
  chunk_count = math.ceil(token_count / token_limit)
  chunk_list = []
  tokenized_chunk_list = []
  for i in range(chunk_count):
    tokenized_chunk = tokens[i*token_limit:(i+1)*token_limit]
    chunk_list.append(tokenizer.decode(tokenized_chunk))
  return chunk_list

#Write chunks to dataframe
tqdm.pandas()
df['Text Chunks'] = df['Article Text'].progress_apply(chunk_contents)
df

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

100%|██████████| 10053/10053 [23:07<00:00,  7.24it/s]


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index,Text Chunks
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"[[CLS] stock report | may 16, 2024 | nysesymbo..."
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN,1,"[[CLS] stock report | may 14, 2024 | nysesymbo..."
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN,2,"[[CLS] stock report | may 11, 2024 | nysesymbo..."
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN,3,"[[CLS] stock report | may 11, 2024 | nysesymbo..."
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN,4,"[[CLS] stock report | may 07, 2024 | nysesymbo..."
...,...,...,...,...,...,...,...,...,...
10048,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN,10048,[[CLS] question and answer operator [ operator...
10049,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10049,[[CLS] question and answer operator [ operator...
10050,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN,10050,[[CLS] question and answer operator [ operator...
10051,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN,10051,[[CLS] question and answer operator [ operator...


### Apply FinBERT Sentiment Analysis
Create Neurtal, Positive, Negative matrix for each chunk

In [ ]:
#split into chunks and create new dataframe
df_chunk = df.explode('Text Chunks').reset_index(drop=True)

#Set NaN Tickers to blanks
df_chunk['Ticker'] = df_chunk['Ticker'].fillna('')
df_chunk

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index,Text Chunks
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"[CLS] stock report | may 16, 2024 | nysesymbol..."
1,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"797 e 1, 804 e 7, 155 2024 1, 536 e 1, 705 e 1..."
2,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,". 12 - 1. 16 uin q1 2024, mro ’ s capex ( $ 60..."
3,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,be relied as such. mro with revenue growth of ...
4,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"texas, e & p. 77024 - 2217 the company ’ s est..."
...,...,...,...,...,...,...,...,...,...
122468,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10052,to the buyer in full. we're not - - there's no...
122469,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10052,we're not - - these investment decisions are n...
122470,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10052,received a distribution this quarter from apln...
122471,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10052,beginning in the third or fourth quarter this ...


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

sentiment_checkpoint = 'ProsusAI/finbert'
softmax = torch.nn.Softmax(dim=1)

######## Sentiment Analysis ############################
finbert = BertForSequenceClassification.from_pretrained(sentiment_checkpoint)#,num_labels=3)
finbert.to(device)
tokenizer = BertTokenizer.from_pretrained(sentiment_checkpoint)

labels = {0:'neutral', 1:'positive',2:'negative'}

def finbert_sentiment(chunk):
  '''
  This function runs the FinBERT sentiment analysis model. This model runs 1
  input at a time because the tokenizer on CPU is the slowest part so batching
  through FinBERT on GPU is negligble time improvement

  Parameters:
  chunk (str): a text chunk from the chunked dataframe

  Returns:
  content_sentiment_list (list of floats): returns a list of the sentiment
  scores
  '''
  with torch.no_grad():
    #content_sentiment_list = []
    inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True)
    inputs.to(device)
    outputs = finbert(**inputs)[0]
    softmax_outputs = softmax(outputs)[0]
    content_sentiment_list = [softmax_outputs[0].item(), softmax_outputs[1].item(), softmax_outputs[2].item()]
  return content_sentiment_list

tqdm.pandas()
df_chunk[['Neutral', 'Positive', 'Negative']] = pd.DataFrame(df_chunk['Text Chunks'].progress_apply(finbert_sentiment).tolist(), index=df_chunk.index)
df_chunk


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

100%|██████████| 122473/122473 [1:14:16<00:00, 27.48it/s]


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index,Text Chunks,Neutral,Positive,Negative
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"[CLS] stock report | may 16, 2024 | nysesymbol...",0.444543,0.043870,0.511587
1,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"797 e 1, 804 e 7, 155 2024 1, 536 e 1, 705 e 1...",0.010882,0.962472,0.026646
2,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,". 12 - 1. 16 uin q1 2024, mro ’ s capex ( $ 60...",0.099641,0.584337,0.316022
3,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,be relied as such. mro with revenue growth of ...,0.018524,0.045159,0.936317
4,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"texas, e & p. 77024 - 2217 the company ’ s est...",0.193506,0.013857,0.792637
...,...,...,...,...,...,...,...,...,...,...,...,...
122468,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10052,to the buyer in full. we're not - - there's no...,0.121257,0.016350,0.862393
122469,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10052,we're not - - these investment decisions are n...,0.089957,0.032235,0.877809
122470,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10052,received a distribution this quarter from apln...,0.058418,0.025884,0.915698
122471,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10052,beginning in the third or fourth quarter this ...,0.140119,0.010999,0.848883


In [ ]:
#Save csv locally
df_chunk = df_chunk[['Unique_ID','Ticker','Source','Date','Article Headline','Text Chunks','URL','Neutral', 'Positive', 'Negative']]
df_chunk.to_csv('finbert_sentiment_data_checkpoint.csv', index=True)

In [ ]:
from google.colab import files
try:
  files.download('finbert_sentiment_data_checkpoint.csv')
except:
  files.download('finbert_sentiment_data_checkpoint.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Merge chunk sentiments into text sentiment

In [ ]:
#Aggregate chunk sentements and classifications back into Article level dataframe
grouped_df = df_chunk.groupby(['Unique_ID','Ticker','Source','Date','Article Headline'])[['Neutral', 'Positive', 'Negative']].mean().reset_index()

grouped_df

,Unique_ID,Ticker,Source,Date,Article Headline,Neutral,Positive,Negative
0,EP-1,CVX,Earnings Call Presentations,2024-04-26,"Chevron Corporation, Q1 2024 Earnings Call, Ap...",0.236644,0.240358,0.522998
1,EP-10,CVX,Earnings Call Presentations,2022-01-28,"Chevron Corporation, Q4 2021 Earnings Call, Ja...",0.270140,0.262477,0.467383
2,EP-100,OXY,Earnings Call Presentations,2021-02-23,"Occidental Petroleum Corporation, Q4 2020 Earn...",0.571726,0.149435,0.278839
3,EP-102,VLO,Earnings Call Presentations,2021-04-22,"Valero Energy Corporation, Q1 2021 Earnings Ca...",0.293977,0.254785,0.451238
4,EP-103,HES,Earnings Call Presentations,2021-04-28,"Hess Corporation, Q1 2021 Earnings Call, Apr 2...",0.462609,0.175901,0.361490
...,...,...,...,...,...,...,...,...
7943,SEC-95017023019312,DVN,SEC Filings,2023-05-09,10-Q,0.061233,0.105453,0.833314
7944,SEC-95017023036885,DVN,SEC Filings,2023-08-02,10-Q,0.047105,0.096601,0.856294
7945,SEC-95017023060838,DVN,SEC Filings,2023-11-08,10-Q,0.068920,0.068996,0.862085
7946,SEC-95017024021781,DVN,SEC Filings,2024-02-28,10-K,0.060758,0.140647,0.798595


In [ ]:
#Drop article text from chunk dataframe
#df_chunk = df_chunk.drop('Article Text', axis=1)
df_chunk['Chunk_ID'] = df_chunk['Unique_ID'] + "-" + df_chunk.index.astype(str)

df_chunk

<ipython-input-11-248f317b3504>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chunk['Chunk_ID'] = df_chunk['Unique_ID'] + "-" + df_chunk.index.astype(str)


,Unique_ID,Ticker,Source,Date,Article Headline,Text Chunks,URL,Neutral,Positive,Negative,Chunk_ID
0,IR-1,MRO,Investment Research,2024-05-16,Marathon Oil Corporation,"[CLS] stock report | may 16, 2024 | nysesymbol...",NaN,0.444543,0.043870,0.511587,IR-1-0
1,IR-1,MRO,Investment Research,2024-05-16,Marathon Oil Corporation,"797 e 1, 804 e 7, 155 2024 1, 536 e 1, 705 e 1...",NaN,0.010882,0.962472,0.026646,IR-1-1
2,IR-1,MRO,Investment Research,2024-05-16,Marathon Oil Corporation,". 12 - 1. 16 uin q1 2024, mro ’ s capex ( $ 60...",NaN,0.099641,0.584337,0.316022,IR-1-2
3,IR-1,MRO,Investment Research,2024-05-16,Marathon Oil Corporation,be relied as such. mro with revenue growth of ...,NaN,0.018524,0.045159,0.936317,IR-1-3
4,IR-1,MRO,Investment Research,2024-05-16,Marathon Oil Corporation,"texas, e & p. 77024 - 2217 the company ’ s est...",NaN,0.193506,0.013857,0.792637,IR-1-4
...,...,...,...,...,...,...,...,...,...,...,...
122468,EQ-342,COP,Earnings Call Q&A,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",to the buyer in full. we're not - - there's no...,NaN,0.121257,0.016350,0.862393,EQ-342-122468
122469,EQ-342,COP,Earnings Call Q&A,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",we're not - - these investment decisions are n...,NaN,0.089957,0.032235,0.877809,EQ-342-122469
122470,EQ-342,COP,Earnings Call Q&A,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",received a distribution this quarter from apln...,NaN,0.058418,0.025884,0.915698,EQ-342-122470
122471,EQ-342,COP,Earnings Call Q&A,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",beginning in the third or fourth quarter this ...,NaN,0.140119,0.010999,0.848883,EQ-342-122471


Write to GitHub Repo

In [ ]:
#Save csv locally
grouped_df.to_csv('/content/sample_data/finbert_sentiment_data.csv', index=True)

# Determine the number of rows for each split
n = len(df_chunk)
split1 = int(n / 3)
split2 = int(2 * n / 3)

# Split the DataFrame
df1_chunk = df_chunk[:split1]
df2_chunk = df_chunk[split1:split2]
df3_chunk = df_chunk[split2:]

df1_chunk[['Chunk_ID','Text Chunks']].to_csv('/content/sample_data/finbert_sentiment_chunkdata_pt1.csv', index=False)
df2_chunk[['Chunk_ID','Text Chunks']].to_csv('/content/sample_data/finbert_sentiment_chunkdata_pt2.csv', index=False)
df3_chunk[['Chunk_ID','Text Chunks']].to_csv('/content/sample_data/finbert_sentiment_chunkdata_pt3.csv', index=False)
df_chunk.to_csv('/content/sample_data/finbert_sentiment_chunkdata.csv', index=True)

In [ ]:
#Copy CSV to GitHub Repository
!cp /content/sample_data/finbert_sentiment_data.csv /content/Financial_Sentiment_LLM/03_Sentiment_Analysis/
!cp /content/sample_data/finbert_sentiment_chunkdata_pt1.csv /content/Financial_Sentiment_LLM/03_Sentiment_Analysis/
!cp /content/sample_data/finbert_sentiment_chunkdata_pt2.csv /content/Financial_Sentiment_LLM/03_Sentiment_Analysis/
!cp /content/sample_data/finbert_sentiment_chunkdata_pt3.csv /content/Financial_Sentiment_LLM/03_Sentiment_Analysis/

In [ ]:
#Commit and Push Changes
!git config --global user.email "bw58@rice.edu"
%cd /content/Financial_Sentiment_LLM/03_Sentiment_Analysis/
!git add finbert_sentiment_data.csv
!git commit -m "Add finbert_sentiment_data.csv"
!git add finbert_sentiment_chunkdata_pt1.csv
!git commit -m "Add finbert_sentiment_chunkdata_ot1.csv"
!git add finbert_sentiment_chunkdata_pt2.csv
!git commit -m "Add finbert_sentiment_chunkdata_pt2.csv"
!git add finbert_sentiment_chunkdata_pt3.csv
!git commit -m "Add finbert_sentiment_chunkdata_pt3.csv"
!git push origin main

/content/Financial_Sentiment_LLM/03_Sentiment_Analysis
[main b13b62c] Add finbert_sentiment_data.csv
 1 file changed, 7949 insertions(+), 208877 deletions(-)
 rewrite 03_Sentiment_Analysis/finbert_sentiment_data.csv (98%)
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 349.57 KiB | 4.85 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 loc

In [ ]:
#Remove github clone
#!rm -rf /content/CVX_Rice_project/

In [ ]:
from google.colab import runtime
runtime.unassign()